In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

![DSL](https://miro.medium.com/max/2002/0*2mgI13cKNCWByVMB)

# Início

## Costa Rican Household Poverty Level Prediction

O objetivo da competição "Costa Rican Household Poverty Level Prediction" é desenvolver um modelo de aprendizado de máquina que possa prever o nível de pobreza das famílias usando características individuais e familiares.

## Objetivo

O objetivo é prever a pobreza ao nível  familiar. Recebemos dados em nível individual, com cada indivíduo tendo características únicas, mas também informações sobre sua família. No conjunto de dados para a tarefa, teremos que realizar algumas agregações dos dados individuais para cada família. Além disso, temos que fazer uma previsão para cada indivíduo no conjunto de teste, mas apenas os chefes de família são usados na pontuação, o que significa que queremos prever a pobreza em uma família.

> Observação importante: embora todos os membros de uma família devam ter o mesmo rótulo nos dados de treinamento, há erros onde os indivíduos na mesma casa têm rótulos diferentes.
> Nestes casos, somos orientados a usar o rótulo do chefe de cada família, que pode ser identificado pelas linhas onde parentesco1 == 1.0. discussão principal da competição

Os valores alvo representam os níveis de pobreza da seguinte forma:

1. pobreza extrema
2. pobreza moderada
3. famílias vulneráveis
4. famílias não vulneráveis

As explicações para todas as 143 colunas podem ser encontradas na documentação da competição, mas algumas merecem destaque:

- **Id**: um identificador único para cada indivíduo, este não deve ser um recurso que usamos!
- **idhogar**: um identificador único para cada família. Esta variável não é um recurso, mas será usada para agrupar indivíduos por domicílio, pois todos os indivíduos em um domicílio terão o mesmo identificador.
- **parentesco1**: indica se essa pessoa é o chefe da família.
- **Target**: o rótulo, que deve ser igual para todos os membros de uma família

## Metodologia
1. Explorar dados e realizar visualização de dados.
2. Verificar valores ausentes e proceder com a correção adequada.
3. Executar a **feature engineering**.
4. Dimensionar recursos numéricos e caso necessário, execute **One HOT Encoding** de recursos categóricos.
5. Selecionar alguns estimadores para o trabalho trabalho.


> * XGBoost


# Entendimento do negócio/problema

Os dados para esta competição são fornecidos em dois arquivos: train.csv e test.csv. O conjunto de treinamento tem 9557 linhas e 143 colunas, enquanto o conjunto de teste tem 23856 linhas e 142 colunas. Cada linha representa um indivíduo e cada coluna é um recurso, exclusivo para o indivíduo ou para a família do indivíduo. O conjunto de treinamento tem uma coluna adicional, TARGET, que representa o nível de pobreza em uma escala de 1 a 4 e é o rótulo da competição. Um valor de 1 é a pobreza mais extrema.

Este é um problema de aprendizado de máquina de classificação multi-classe supervisionado:

Supervisionado: fornecido com os rótulos dos dados de treinamento
Classificação multiclasse: os rótulos são valores discretos com 4 classes

# Aquisição de dados

### Fonte de dados

In [ ]:
# Carregar dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

### Manipulação, exploração e limpeza de dados

In [ ]:
# Juntar dataframes
df_all = df.append(test)
df_all.shape

In [ ]:
# Visualizar dados
df_all.info()

In [ ]:
# Aumentar número de colunas a serem exibidas
df_all.info(max_cols=145)

In [ ]:
# Utilizar pandas para definir valores inf como na
pd.set_option('mode.use_inf_as_na', True)

**edjefa** e **edjefe**

Dicionário de dados define edjefa e edjefe da seguinte forma:

* edjefe - years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

* edjefe - years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

In [ ]:
# Analisar dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisar dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Transformar 'yes' em 1 e 'no' em 0 nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Identificar colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Transformar 'yes' em 1 e 'no' em 0 na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Identificar colunas do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizar utilizando comando info
df_all.info(max_cols=145)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

**Features que possuem missing values**
* v2a1 = valor aluguel mensal
* v18q1 = quantidade de tablets que os proprietários da casa possuem
* rez_esc = anos antes da escola
* meaneduc = média ded anos de educação nos adultos
* SQBmeaned = quadrado da média dos anos de educação dos adultos

In [ ]:
# Verificar os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Alerar para -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

A escolha para preenchimento dos valores nulos por -1 não foi feita ao acaso. A intenção é não tentar excluir essa coluna, bem como não perder dados. Como o algoritmo usado será Random Forest, o uso de -1 (outlier) será para "forçar" a árvore a dispensar os valores -1.

In [ ]:
# Análise de v18q (relacao com v18q1)
df_all['v18q'].value_counts()

A quantidade de valores nulos na feature v18q (possui tablet), que é binária, coincide com o valor de missing values da feature quantidade de tablets. Deta forma, substuímos valores nulos por 0.

In [ ]:
# Análise de rez_esc (relacao com v18q1)
df_all['rez_esc'].value_counts()

In [ ]:
# Prencher com -1 os valores nulos de rez_esc (estratégia do outlier)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Verificar valores nulos de meaneduc
print('Porcentagem de Valores Nulos:',(df_all['meaneduc'].isnull().sum() / len(df_all))*100)
print('Quantidade de Valores Nulos:',df_all['meaneduc'].isnull().sum() )

In [ ]:
# Aferir média e mediana de meaneduc
df_all.meaneduc.mean(), df_all.meaneduc.median()

In [ ]:
# Preencher os valores nulos de meaneduc com o valor de 9 anos de estudos (entre média e mediana), ou seja, uma tendência geral dos dados
df_all['meaneduc'].fillna(9, inplace=True)

In [ ]:
# Verificar valores nulos de SQBmeaned
print('Porcentagem de Valores Nulos:',(df_all['SQBmeaned'].isnull().sum() / len(df_all))*100)

In [ ]:
# Preencher com -1 os valores nulos de SQBmeaned (estratégia do outlier)
df_all['SQBmeaned'].fillna(-1, inplace=True)

In [ ]:
#Visualização dados pós manipulação, exploração e limpeza
df_all.info(max_cols=145)

In [ ]:
df_all.isnull().sum().sort_values()

# Modelagem

In [ ]:
# Separar colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]
train.shape, test.shape

In [ ]:
# Plotar histograma da variável "Target"
sns.histplot(data=train, x="Target", bins = 4)
plt.show()

In [ ]:
# Verificar valores absolutos
train['Target'].value_counts()

In [ ]:
# Verificar porcentagens
train['Target'].value_counts(normalize=True)

É necessário a utilização de técnica para balancemaneto das classes minotitárias.

### Feature Engineering

In [ ]:
# Dividir dataset treino em X,y
X, y = train[feats], train[['Target']]

In [ ]:
# Importar biblioteca
from imblearn.over_sampling import RandomOverSampler

# Realizar over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X,y)

# Verificar dados
y_ros['Target'].value_counts()

### Treinamento do modelo com XGBoost após "feature Engineering" (oversampling)

In [ ]:
# Utilizar XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=250, learning_rate=0.09, random_state=42)

# Treinar
xgb.fit(X_ros, y_ros)

### Avaliação do modelo

In [ ]:
# Verificar previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Prever o Target de teste usando modelo treinado
test['Target'] = xgb.predict(test[feats]).astype(int)

# Deployment

In [ ]:
# Criar arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
fig=plt.figure(figsize=(15, 20))

# Avaliar importancia de cada coluna
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()